In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
%matplotlib inline
import seaborn as sns; sns.set_theme(color_codes=True)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
import math

In [2]:
#define current city
city = 'newberg'

In [25]:
# read in dara/jon-miket-validates
df1 = pd.read_csv('../../../data/shared/jon-mikey-validated-labels-correct.csv')
df2 = pd.read_csv('../../../data/shared/jon-mikey-validated-labels-incorrect.csv')

In [26]:
#select current city data
df1 = df1[df1['city'] == city]
df2 = df2[df2['city'] == city]

In [27]:
#select only label_id
df1 = df1[['label_id']]
df2 = df2[['label_id']]

In [28]:
#create a new column for correct/incorrect called ground_truth
df1['ground_truth'] = 1
df2['ground_truth'] = 0

In [29]:
#combine the two dataframes
df = pd.concat([df1, df2])

In [30]:
df.sort_values(by=['label_id'], inplace=True)

In [31]:
df

,label_id,ground_truth
25962,8,1
25963,9,1
25964,10,1
25965,11,1
25966,12,1
...,...,...
6781,17672,0
28344,17674,1
28345,17676,1
28346,17688,1


In [32]:
#read in labels
labels = gpd.read_file('../../../data/{}/processed-labels/labels_raw_{}/labels_raw_{}.shp'.format(city, city, city))
labels

,label_id,gsv_panora,label_type,severity,correct,has_valida,expired,high_quali,gsv_pano_1,pano_width,pano_heigh,pano_x,pano_y,zoom,heading,pitch,camera_hea,camera_pit,user_id,lat,lng,geometry
0,145,0R2JTLibSCNsNb7aGNUzWg,NoSidewalk,NaN,None,0,1,1,0R2JTLibSCNsNb7aGNUzWg,13312.0,6656.0,9123,3863,1,319.375000,-19.062500,269.813477,0.678093,d363619a-53e1-4ef4-a541-bb0278cae7c3,45.317650,-122.952904,POINT (-122.95290 45.31765)
1,18134,Z6Ja9-4wJxpQZPEnn8KSXA,NoSidewalk,4.0,None,0,0,1,Z6Ja9-4wJxpQZPEnn8KSXA,13312.0,6656.0,6644,3744,3,326.437500,-10.906250,323.408020,1.379303,bb78d1ab-85d6-4620-a1de-396a5785080a,45.313820,-122.964005,POINT (-122.96400 45.31382)
2,151,oqg9ULHZul23xfU6DwTRNg,NoSidewalk,NaN,None,0,0,1,oqg9ULHZul23xfU6DwTRNg,13312.0,6656.0,7788,3776,2,285.258942,-11.508928,268.211182,0.926697,d363619a-53e1-4ef4-a541-bb0278cae7c3,45.317616,-122.953598,POINT (-122.95360 45.31762)
3,153,WWRq8hx_qzBmTiSkSBjEqQ,NoSidewalk,NaN,None,0,1,1,WWRq8hx_qzBmTiSkSBjEqQ,13312.0,6656.0,4663,3709,2,223.651779,-8.741072,268.896179,-0.547432,d363619a-53e1-4ef4-a541-bb0278cae7c3,45.317421,-122.954025,POINT (-122.95403 45.31742)
4,154,WWRq8hx_qzBmTiSkSBjEqQ,NoSidewalk,NaN,None,0,1,1,WWRq8hx_qzBmTiSkSBjEqQ,13312.0,6656.0,7268,3600,2,279.008942,-9.455358,268.896179,-0.547432,d363619a-53e1-4ef4-a541-bb0278cae7c3,45.317600,-122.954185,POINT (-122.95419 45.31760)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16842,8222,5KKnsHQgODUBbIRZYiVbqg,NoSidewalk,NaN,None,0,0,1,5KKnsHQgODUBbIRZYiVbqg,13312.0,6656.0,256,3757,2,225.142853,-19.375000,22.380564,-3.511421,fc36440f-38b7-4060-892a-65fedd77f651,45.303329,-122.925797,POINT (-122.92580 45.30333)
16843,8272,mdbgEoi-XTfZGiKqcWMlqw,NoSidewalk,NaN,None,0,1,1,mdbgEoi-XTfZGiKqcWMlqw,3328.0,1664.0,2697,986,1,114.982140,-15.312500,359.096985,-9.832024,fc36440f-38b7-4060-892a-65fedd77f651,45.299858,-122.925591,POINT (-122.92559 45.29986)
16844,8281,BtyhcErvfI4z31FisBjeQg,NoSidewalk,NaN,None,0,0,1,BtyhcErvfI4z31FisBjeQg,13312.0,6656.0,7561,3633,1,43.812500,-1.875000,46.910660,-2.319252,fc36440f-38b7-4060-892a-65fedd77f651,45.305443,-122.926605,POINT (-122.92661 45.30544)
16845,16595,--gG93FA53Lkv0H31-9Ruw,CurbRamp,NaN,1,1,0,1,--gG93FA53Lkv0H31-9Ruw,13312.0,6656.0,1748,3983,1,142.982147,-11.625000,270.257080,-1.495285,fc36440f-38b7-4060-892a-65fedd77f651,45.299377,-122.956367,POINT (-122.95637 45.29938)


In [33]:
#select label_ids in df are in labels
df = df[df['label_id'].isin(labels['label_id'])]
df

,label_id,ground_truth
25962,8,1
25963,9,1
25964,10,1
25965,11,1
25966,12,1
...,...,...
6781,17672,0
28344,17674,1
28345,17676,1
28346,17688,1


In [35]:
#check values of ground_truth
df['ground_truth'].value_counts()

1    2364
0     431
Name: ground_truth, dtype: int64

In [36]:
#save the ground truth labels to a csv
df.to_csv('../../../data/{}/features/{}_ground_truth_labels.csv'.format(city,city), index=False)